In [1]:
# Initial imports
#import os
#import requests
import pandas as pd
from pathlib import Path
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
#from MCForecastTools import MCSimulation
#import warnings
#warnings.filterwarnings("ignore")

#%matplotlib inline

In [2]:
# Load .env enviroment variables
load_dotenv()

True

In [29]:
# read in portfolio.csv format date,ticker,shares,share_price, clean data, set date as index,
portfolio_df = pd.read_csv(Path('portfolio.csv'), index_col=0, parse_dates=True, infer_datetime_format=True)
portfolio_df
#list(portfolio_df.columns) 


,symbol,no_of_shares,share_price
order_date,,,
2020-03-16,AMZN,59,84.46
2020-03-16,AAPL,83,60.50
2020-03-16,MSFT,37,135.42
2020-03-16,TSLA,169,29.67
2021-03-05,AMZN,33,150.02
2022-03-15,AAPL,320,155.09
2022-01-24,MSFT,170,296.37
2022-03-14,TSLA,200,255.46


In [30]:
#  add profit/loss column, (takes the number of shares * purchase share_price) - current prices
portfolio_df['order_total'] = portfolio_df['no_of_shares']*portfolio_df['share_price']
#clean_portfolio_df

#join_data_columns = pd.concat([portfolio_df, clean_portfolio_df], axis= 'columns', join="inner")
#join_data_columns.head()

#join_data_columns.drop(join_data_columns.filter(regex="0"),axis=1, inplace=True)
#join_data_columns.head()

print(portfolio_df)

           symbol  no_of_shares  share_price  order_total
order_date                                               
2020-03-16   AMZN            59        84.46      4983.14
2020-03-16   AAPL            83        60.50      5021.50
2020-03-16   MSFT            37       135.42      5010.54
2020-03-16   TSLA           169        29.67      5014.23
2021-03-05   AMZN            33       150.02      4950.66
2022-03-15   AAPL           320       155.09     49628.80
2022-01-24   MSFT           170       296.37     50382.90
2022-03-14   TSLA           200       255.46     51092.00


In [31]:
# get current prices. use api to get current price

API_KEY = os.getenv('ALPACA_API_KEY')

API_SECRET_KEY = os.getenv('ALPACA_SECRET_KEY')

type(API_KEY)

# Create the Alpaca API object
alpaca = tradeapi.REST(
    API_KEY,
    API_SECRET_KEY,
    api_version='v2'
)

# Format current date as ISO format NEED TO AUTO GRAB TODAY'S DATE
today = pd.Timestamp("2022-12-01", tz='America/New_York').isoformat()

# Set the tickers
tickers = portfolio_df['symbol'].unique()

# Set timeframe to "1Day" for Alpaca API
timeframe = "1Day"

stock_price_df = alpaca.get_bars(
    tickers,
    timeframe,
    start=today,
    end=today
).df

stock_price_df.head(10)

portfolio_df.reset_index(inplace=True)

In [32]:
# concat current prices to portfolio 
#current_close_df = pmerge([portfolio_df, stock_price_df], axis= 'columns', join="left")
#current_close_df.head()
from datetime import *


current_price_df = portfolio_df.merge(stock_price_df, how='left', on='symbol')
current_price_df['order_date'] = pd.to_datetime(current_price_df['order_date']).dt.date
current_price_df['current_value'] = current_price_df['no_of_shares']*current_price_df['close']
date_check = date.today() - current_price_df['order_date'] 
current_price_df['bought_days_from_today'] = date_check
current_price_df['bought_days_from_today'] = current_price_df['bought_days_from_today'].dt.days
current_price_df['tax_type'] = ''
current_price_df


#gains = current_price_df['bought_days_from_today']

#for i in gains:
 #   if gains > 365:
  #      print('true')
   # elif 
    #    print('false')

#print(current_price_df['bought_days_from_today'])
#if current_price_df['bought_days_from_today']>365:
#    print ('True')
#else:
 #    print ('not true')

    # > 365:
    #print('its true')

#print(current_price_df['order_date'])


#print(current_price_df['order_date'])

#date_check = date.today() - current_price_df['order_date'] 

#print(date_check)


#print(date.today())
#if current_price_df['date'] > 

#add total value column to dataframe profit/loss column
#portfolio.head()

,order_date,symbol,no_of_shares,share_price,order_total,open,high,low,close,volume,trade_count,vwap,current_value,bought_days_from_today,tax_type
0,2020-03-16,AMZN,59,84.46,4983.14,96.99,97.23,94.9200,95.50,68487969,566475,95.943172,5634.50,994,
1,2020-03-16,AAPL,83,60.50,5021.50,148.21,149.13,146.6100,148.31,71250381,525985,148.105003,12309.73,994,
2,2020-03-16,MSFT,37,135.42,5010.54,253.87,256.12,250.9156,254.69,26054508,352799,254.358458,9423.53,994,
3,2020-03-16,TSLA,169,29.67,5014.23,197.08,198.92,191.8000,194.70,80104611,855150,195.127022,32904.30,994,
4,2021-03-05,AMZN,33,150.02,4950.66,96.99,97.23,94.9200,95.50,68487969,566475,95.943172,3151.50,640,
5,2022-03-15,AAPL,320,155.09,49628.80,148.21,149.13,146.6100,148.31,71250381,525985,148.105003,47459.20,265,
6,2022-01-24,MSFT,170,296.37,50382.90,253.87,256.12,250.9156,254.69,26054508,352799,254.358458,43297.30,315,
7,2022-03-14,TSLA,200,255.46,51092.00,197.08,198.92,191.8000,194.70,80104611,855150,195.127022,38940.00,266,


In [33]:
current_price_df.loc[current_price_df['bought_days_from_today'] >=365, 'tax'] = 'LT'
current_price_df.loc[current_price_df['bought_days_from_today'] < 365, 'tax'] = 'ST'


#current_price_df['tax_type'] = (current_price_df['bought_days_from_today'] >= 365).all()
   # current_price_df['tax_type'] = 'LT'
#else :
 #   current_price_df['tax_type'] = 'ST'

current_price_df

,order_date,symbol,no_of_shares,share_price,order_total,open,high,low,close,volume,trade_count,vwap,current_value,bought_days_from_today,tax_type,tax
0,2020-03-16,AMZN,59,84.46,4983.14,96.99,97.23,94.9200,95.50,68487969,566475,95.943172,5634.50,994,,LT
1,2020-03-16,AAPL,83,60.50,5021.50,148.21,149.13,146.6100,148.31,71250381,525985,148.105003,12309.73,994,,LT
2,2020-03-16,MSFT,37,135.42,5010.54,253.87,256.12,250.9156,254.69,26054508,352799,254.358458,9423.53,994,,LT
3,2020-03-16,TSLA,169,29.67,5014.23,197.08,198.92,191.8000,194.70,80104611,855150,195.127022,32904.30,994,,LT
4,2021-03-05,AMZN,33,150.02,4950.66,96.99,97.23,94.9200,95.50,68487969,566475,95.943172,3151.50,640,,LT
5,2022-03-15,AAPL,320,155.09,49628.80,148.21,149.13,146.6100,148.31,71250381,525985,148.105003,47459.20,265,,ST
6,2022-01-24,MSFT,170,296.37,50382.90,253.87,256.12,250.9156,254.69,26054508,352799,254.358458,43297.30,315,,ST
7,2022-03-14,TSLA,200,255.46,51092.00,197.08,198.92,191.8000,194.70,80104611,855150,195.127022,38940.00,266,,ST


In [36]:
# display blanace of each tax type and ask user to input dollar total of withdrawal requested, put amount in column "amount_to_withdraw"
LT_portfolio_df = current_price_df.loc[current_price_df['tax'] == 'LT']
LT_portfolio_balance = LT_portfolio_df['current_value'].sum()
LT_portfolio_inital_investment = LT_portfolio_df['order_total'].sum()
LT_taxable_amount = LT_portfolio_balance - LT_portfolio_inital_investment
display(LT_portfolio_df)
display(ST_portfolio_df)

print(f"the value of your long term gains taxable portfolio is  $ {LT_portfolio_balance}")
print(f"your initial investment is  $ {LT_portfolio_inital_investment}")
print(f"the taxable difference is  $ {LT_portfolio_balance} - {LT_portfolio_inital_investment} = {LT_taxable_amount} ")

ST_portfolio_df = current_price_df.loc[current_price_df['tax'] == 'ST']
ST_portfolio_balance = ST_portfolio_df['current_value'].sum()
ST_portfolio_inital_investment = ST_portfolio_df['order_total'].sum()
ST_taxable_amount = ST_portfolio_balance - ST_portfolio_inital_investment
total_portfolio_value = LT_portfolio_balance + ST_portfolio_balance





print(f"the value of your short term gains taxable portfolio is  $ {ST_portfolio_balance}")
print(f"your initial investment is  $ {ST_portfolio_inital_investment}")
print(f"the taxable difference is  $ {ST_portfolio_balance} - {ST_portfolio_inital_investment} = {ST_taxable_amount} ")
print(f"total portfolio value is $ {total_portfolio_value}")


,order_date,symbol,no_of_shares,share_price,order_total,open,high,low,close,volume,trade_count,vwap,current_value,bought_days_from_today,tax_type,tax
0,2020-03-16,AMZN,59,84.46,4983.14,96.99,97.23,94.9200,95.50,68487969,566475,95.943172,5634.50,994,,LT
1,2020-03-16,AAPL,83,60.50,5021.50,148.21,149.13,146.6100,148.31,71250381,525985,148.105003,12309.73,994,,LT
2,2020-03-16,MSFT,37,135.42,5010.54,253.87,256.12,250.9156,254.69,26054508,352799,254.358458,9423.53,994,,LT
3,2020-03-16,TSLA,169,29.67,5014.23,197.08,198.92,191.8000,194.70,80104611,855150,195.127022,32904.30,994,,LT
4,2021-03-05,AMZN,33,150.02,4950.66,96.99,97.23,94.9200,95.50,68487969,566475,95.943172,3151.50,640,,LT


,order_date,symbol,no_of_shares,share_price,order_total,open,high,low,close,volume,trade_count,vwap,current_value,bought_days_from_today,tax_type,tax
5,2022-03-15,AAPL,320,155.09,49628.8,148.21,149.13,146.6100,148.31,71250381,525985,148.105003,47459.2,265,,ST
6,2022-01-24,MSFT,170,296.37,50382.9,253.87,256.12,250.9156,254.69,26054508,352799,254.358458,43297.3,315,,ST
7,2022-03-14,TSLA,200,255.46,51092.0,197.08,198.92,191.8000,194.70,80104611,855150,195.127022,38940.0,266,,ST


the value of your long term gains taxable portfolio is  $ 63423.56
your initial investment is  $ 24980.07
the taxable difference is  $ 63423.56 - 24980.07 = 38443.49 
the value of your short term gains taxable portfolio is  $ 129696.5
your initial investment is  $ 151103.7
the taxable difference is  $ 129696.5 - 151103.7 = -21407.20000000001 
total portfolio value is $ 193120.06


In [ ]:
# ask user for yearly income to determine tax bracket. put amount in column "taxable_income"

In [ ]:
# convert "taxable_income" to percentage of taxes owed from tax bracket into column "tax_pct"

In [ ]:
# ask user for 2 letter state abbreviation saved in var "user_state"

In [ ]:
# look up state tax by user_state. add state_tax to tax_pct

In [ ]:
# calculate from profit/loss column which shares to sell that has the closest total to zero. 

In [ ]:
# save the net total in a new column "tax_liability", additional new column "short_term/long_term_gains

In [ ]:
# display the ticker and number of shares to sell to get to desired amount_to_withdraw

In [ ]:
# display new tax bill by taking ticker sold value * tax_pct